# Model

> Scripts to train the models

In [48]:
#| default_exp model

In [49]:
#| export
#| hide
# from tensorflow.keras.optimizers import Adam, SGD

from tensorflow import keras
import tensorflow as tf
import tsgm.models
import matplotlib.pyplot as plt

In [50]:
#| hide
class BetaVAE(keras.Model):
    """
    beta-VAE implementation for unlabeled time series.
    """
    def __init__(self, encoder: keras.Model, decoder: keras.Model, beta: float = 1.0, **kwargs) -> None:
        super(BetaVAE, self).__init__(**kwargs)
        self.beta = beta
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self._seq_len = self.decoder.output_shape[1]
        self.latent_dim = self.decoder.input_shape[1]

    @property
    def metrics(self) -> list:
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, X: tsgm.types.Tensor) -> tsgm.types.Tensor:
        z_mean, _, _ = self.encoder(X)
        x_decoded = self.decoder(z_mean)
        if len(x_decoded.shape) == 1:
            x_decoded = x_decoded.reshape((1, -1))
        return x_decoded

    def _get_reconstruction_loss(self, X: tsgm.types.Tensor, Xr: tsgm.types.Tensor) -> float:
        reconst_loss = tsgm.utils.reconstruction_loss_by_axis(X, Xr, axis=0) +\
            tsgm.utils.reconstruction_loss_by_axis(X, Xr, axis=1) +\
            tsgm.utils.reconstruction_loss_by_axis(X, Xr, axis=2)
        return reconst_loss

    def train_step(self, data: tsgm.types.Tensor) -> dict:
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = self._get_reconstruction_loss(data, reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data: tsgm.types.Tensor) -> dict:
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        reconstruction_loss = self._get_reconstruction_loss(data, reconstruction)
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = reconstruction_loss + kl_loss

        # Ensure losses are scalar values
        total_loss = tf.reduce_mean(total_loss)
        reconstruction_loss = tf.reduce_mean(reconstruction_loss)
        kl_loss = tf.reduce_mean(kl_loss)

        return {
            "val_loss": total_loss,
            "val_reconstruction_loss": reconstruction_loss,
            "val_kl_loss": kl_loss,
        }

In [ ]:
import os
import tsgm

def get_model(params, data_path):
    model_name = params['model_name']
    data_used = os.path.splitext(os.path.basename(data_path))[0]

    if model_name == 'vae_conv5':
        # Accessing model configuration from the zoo using parameters from the dictionary
        architecture = tsgm.models.zoo[model_name](
            seq_len=params['seq_len'], 
            feat_dim=params['feature_dim'], 
            latent_dim=params['latent_dim']
        )

        # Extracting encoder and decoder from the architecture
        encoder, decoder = architecture.encoder, architecture.decoder

        # Build the VAE
        vae = tsgm.models.cvae.BetaVAE(encoder, decoder)
        vae.compile(optimizer=params['optimizer']['name'], learning_rate=params['optimizer']['learning_rate'])
        return vae

    elif model_name == 'timeGAN':
        model = tsgm.models.timeGAN.TimeGAN(
            seq_len=params['seq_len'],
            module="gru",
            hidden_dim=24,
            n_features=params['feature_dim'],
            n_layers=3,
            batch_size=params['batch_size'],
            gamma=1.0,
        )
        # .compile() sets all optimizers to Adam by default
        model.compile(optimizer=params['optimizer']['name'], learning_rate=params['optimizer']['learning_rate'])
        return model

    else:
        raise ValueError(f"Unsupported model_name: {model_name}")


In [51]:
#| export
def get_model(params):
    model_name = params['model_name']

    if model_name == 'vae_conv5':
        # Accessing model configuration from the zoo using parameters from the dictionary
        architecture = tsgm.models.zoo[model_name](
            seq_len=params['seq_len'], 
            feat_dim=params['feature_dim'], 
            latent_dim=params['latent_dim']
        )

        # Extracting encoder and decoder from the architecture
        encoder, decoder = architecture.encoder, architecture.decoder

        # Build the VAE
        # vae = BetaVAE(encoder, decoder)
        vae = tsgm.models.cvae.BetaVAE(encoder, decoder)
        return vae

    elif model_name == 'timeGAN':
        model = tsgm.models.timeGAN.TimeGAN(
            seq_len=params['seq_len'],
            module="gru",
            hidden_dim=24,
            n_features=params['feature_dim'],
            n_layers=3,
            batch_size=params['batch_size'],
            gamma=1.0,
        )
        # .compile() sets all optimizers to Adam by default
        model.compile(optimizer=params['optimizer']['name'], learning_rate=params['optimizer']['learning_rate'])
        return model

    else:
        raise ValueError(f"Unsupported model_name: {model_name}")

In [52]:
#| export
def get_optimizer(optimizer_config):
    name = optimizer_config['name'].lower()
    if name == 'adam':
        return keras.optimizers.Adam(learning_rate=optimizer_config.get('learning_rate', 0.001))
    elif name == 'sgd':
        return keras.optimizers.SGD(learning_rate=optimizer_config.get('learning_rate', 0.01))
    # Add additional optimizers as needed
    raise ValueError("Unsupported optimizer: {}".format(optimizer_config['name']))

In [53]:
#| export
def extract_plot_and_return_metrics(history, validation=True):
    """
    Extracts the metrics from the training history, plots the training and validation loss over epochs if validation is True, and returns the metrics.
    
    Parameters:
    - history: History object returned by model.fit().
    - validation: Boolean flag to control whether to extract and plot validation metrics.
    
    Returns:
    - metrics: Dictionary containing the final training and validation metrics.
    """
    metrics = {}

    # Extract training metrics from the training history and add to the metrics dictionary
    metrics['loss'] = history.history['loss'][-1]
    metrics['reconstruction_loss'] = history.history['reconstruction_loss'][-1]
    metrics['kl_loss'] = history.history['kl_loss'][-1]
    
    if validation:
        if 'val_loss' in history.history:
            metrics['val_loss'] = history.history['val_loss'][-1]
        if 'val_reconstruction_loss' in history.history:
            metrics['val_reconstruction_loss'] = history.history['val_reconstruction_loss'][-1]
        if 'val_kl_loss' in history.history:
            metrics['val_kl_loss'] = history.history['val_kl_loss'][-1]

    # Get the number of epochs
    num_epochs = len(history.history['loss'])

    # Plot metrics
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, history.history['loss'], label='Training loss')
    if validation:
        plt.plot(epochs, history.history['val_loss'], label='Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    return metrics


In [54]:
#| hide
import nbdev; nbdev.nbdev_export()